In [1]:
# %%capture --no-stderr
# !pip install git+https://github.com/openai/CLIP timm

In [2]:
import torch
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm

from omegaconf import OmegaConf
from igniter.builder import build_engine
from igniter.main import get_full_config

from fsl.models.meta_arch import build_sam_fsod
from fsl.models.meta_arch import *

plt.rcParams['figure.figsize'] = [10, 8]

In [3]:
config_file = '../configs/devit/resnet_trainval_30shot.yaml'
config_file = '../configs/devit/devit_dinov2_trainval_xshot.yaml'

cfg = get_full_config(config_file)
# cfg = OmegaConf.load(config_file)
# print(OmegaConf.to_yaml(cfg))

cfg.datasets.dataloader.batch_size = 1
cfg.datasets.dataloader.num_workers = 1
cfg.datasets.dataloader.shuffle=False
cfg.options.train = False
cfg.options.eval = True
# cfg.build.resnet_fsod.weights="s3://sr-shokunin/perception/models/fsl/resnet_vitb_coco17_30shot/model_0000001.pt"
# cfg.build.sam_fsod.weights = "s3://sr-shokunin/perception/models/fsl/sam_vitb_coco17_30shot/model_0000005.pt"
# cfg.build.dinov2_fsod.weights = "s3://sr-shokunin/perception/models/fsl/dinov2_vitb_coco17_5shot/model_0000009.pt"

# cfg.build.model = "dinov2_fsod"

engine = build_engine(cfg)

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-15 21:35:49,697 [attention.py:25] WARNING: xFormers not available
2023-11-15 21:35:49,698 [block.py:32] WARNING: xFormers not available
2023-11-15 21:35:50,735 [builder.py:255] WARNING: # TODO: Remove hardcoded name and replace with registry based
/root/krishneel/Documents/github/igniter/igniter/datasets/s3_dataset.py:55: UserWarning: Target transforms is not yet implemented
  warnings.warn('Target transforms is not yet implemented')


loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


/usr/local/lib/python3.10/dist-packages/ignite/contrib/handlers/tqdm_logger.py:126: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
engine._model.eval().to(torch.float16)

FSOD(
  (mask_generator): DinoV2Patch(
    (backbone): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-11): 12 x NestedTensorBlock(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): MemEffAttention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=3072, out_features=768, b

In [5]:
size = max([len(iid) for iid in engine._model.classifier._all_cids]) + 1
gt_labels = lambda indices: [engine._model.classifier._all_cids[i] for i in indices]

def accuracy(scores, gt_names, topk = 1):
    _, indices = torch.topk(scores, topk, dim=1)
    pred_labels = [gt_labels(index) for index in indices]

    count = 0.0
    for gt_name, labels in zip(gt_names, pred_labels):
        count += 1 if gt_name in labels else 0
    return count / len(gt_names)

def print_labels(scores, targets, k = 2):
    gt_names = targets[0]['gt_proposal'].labels
    _, indexes = torch.topk(scores, k, dim=1)
    
    for name, i in zip(gt_names, indexes.cpu().numpy()):
        string = name.ljust(size) + " | " + "".join([n.ljust(size) for n in gt_labels(i)])
        print(string)

In [6]:
acc_top1, acc_top5 = 0.0, 0.0
progress_bar = tqdm(total=len(engine._dataloader), desc="Processing", position=0, leave=True)

for k, (images, targets) in enumerate(engine._dataloader, 1):
    with torch.no_grad():
        output, _ = engine._model(images, targets)
    scores = output['scores']
    # gt_names = targets[0]['gt_proposal'].labels
    gt_names = [target['gt_proposal'].labels for target in targets]
    gt_names = [item for sublist in gt_names for item in sublist]

    t1 = accuracy(scores, gt_names, topk=1)
    t5 = accuracy(scores, gt_names, topk=5)

    acc_top1 += (t1 - acc_top1) / k
    acc_top5 += (t5 - acc_top5) / k

    # print("\nImage ID: ", targets[0]['gt_proposal'].image_id)
    # print_labels(scores, targets, k=5)
    progress_bar.set_description(f"Top 1: {acc_top1:.3f} | Top 5 {acc_top5:.3f}")
    progress_bar.update(1)

progress_bar.close()

print("\nTop 1: ", acc_top1, "Top 5: ", acc_top5)

Processing:   0%|                                                                                     | 0/4167 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Top 1: 0.801 | Top 5 0.893: 100%|████████████████████████████████████████████████████████| 4167/4167 [1:47:24<00:00,  1.55s/it]


Top 1:  0.8010184982734054 Top 5:  0.8928020449343591


In [7]:
# Top 1:  0.43006454187764404 Top 5:  0.659023426230131
# Top 1:  0.4290284612805177 Top 5:  0.659921149440774
